In [125]:
from langgraph.checkpoint.memory import MemorySaver
from typing import Annotated
from langchain_groq import ChatGroq
from langchain.utilities import WikipediaAPIWrapper
from langchain_core.messages import BaseMessage
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START,END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode,tools_condition
import os
from langchain_community.tools import WikipediaQueryRun


In [136]:
memory=MemorySaver()
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")


In [137]:
class State(TypedDict):
    messages:Annotated[list, add_messages]
graph_builder=StateGraph(State)
tool=WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
tools=[tool]


In [138]:
llm = ChatGroq(
    model="llama3-70b-8192",
    temperature=0.7,
)
llm_with_tool=llm.bind_tools(tools=tools)


In [139]:
def chatbot(state:State):
    return {"messages":[llm_with_tool.invoke(state["messages"])]}

In [140]:
graph_builder.add_node("chatbot",chatbot)
tool_node=ToolNode(tools=tools)
graph_builder.add_node("tools",tool_node)
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)

graph_builder.add_edge("tools","chatbot")

graph_builder.set_entry_point("chatbot")
graph=graph_builder.compile(checkpointer=memory)

In [141]:
config={"configurable":{"thread_id":"1"}}

In [144]:
userinput=input()
events=graph.stream({
    "messages":[("user",userinput)]},config,stream_mode="values")
# print(events)
for event in events:
        event["messages"][-1].pretty_print()


================================ Human Message =================================

hey
================================== Ai Message ==================================

Hey again! Is there something I can help you with or would you like to chat?


In [147]:
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage  # Import message types
while True:
    userinput=input()
    if userinput == "q" or userinput == "quit":
        break
    events=graph.stream({
    "messages":[("user",userinput)]},config,stream_mode="values")

    for event in events:
        # Filter messages to exclude tool-related messages
        filtered_messages = [
            msg for msg in event["messages"] if isinstance(msg, (HumanMessage, AIMessage))
        ]
        
        # Display only human and AI-generated messages (tool messages are excluded)
    for msg in filtered_messages:
        msg.pretty_print()



================================ Human Message =================================

hey
================================ Human Message =================================

hey
================================== Ai Message ==================================

Hey! How's it going?
================================ Human Message =================================

hey
================================== Ai Message ==================================

Hey again! Is there something I can help you with or would you like to chat?
================================ Human Message =================================

hey
================================ Human Message =================================

hey
================================== Ai Message ==================================

You're really sticking to "hey" today, aren't you? Would you like to talk about something in particular or just keep the "hey" train going?
================================ Human Message =================================

hey
